# KoNLPy 실습

## 실습: 네이버 영화 평점 데이터 감성분석

### 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import re

In [23]:
train_df = pd.read_csv('./필요 데이터/NAVER_movie/ratings_train.txt', sep = '\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [24]:
test_df = pd.read_csv('./필요 데이터/NAVER_movie/ratings_test.txt', sep = '\t')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


### EDA

In [25]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

- 고르게 분포되어 있음

In [26]:
test_df['label'].value_counts()

1    25173
0    24827
Name: label, dtype: int64

####  null값 확인

In [27]:
import re

In [28]:
train_df['document'].isnull().sum(), test_df['document'].isnull().sum()

(5, 3)

In [29]:
train_df['label'].isnull().sum(), test_df['label'].isnull().sum()

(0, 0)

In [30]:
# null값을 공백으로 바꾸기

train_df['document'] = train_df['document'].fillna(' ')
test_df['document'] = test_df['document'].fillna(' ')

In [31]:
train_df['document'].isnull().sum(), test_df['document'].isnull().sum()

(0, 0)

#### 숫자를 공백으로 변경

In [32]:
train_df['document'] = train_df['document'].apply(lambda x: re.sub('\d+', ' ', x))
test_df['document'] = test_df['document'].apply(lambda x: re.sub('\d+', ' ', x))

In [35]:
train_df.drop('id', axis = 1, inplace = True)
test_df.drop('id', axis = 1, inplace = True)

### 피처벡터화

#### 형태소 토큰화 사용자함수 만들기

In [34]:
from konlpy.tag import Okt

In [37]:
def tw_tokenizer(text):
    okt = Okt()
    token = okt.morphs(text)
    return token

In [40]:
text = train_df['document'][100]
tw_tokenizer(text)

['신카이',
 '마코토',
 '의',
 '작화',
 '와',
 ',',
 '미',
 '유',
 '와',
 '하나',
 '카나',
 '가',
 '연기',
 '를',
 '잘',
 '해줘서',
 '더',
 '대박',
 '이',
 '였다',
 '.']

#### pipeline을 활용한 하이퍼파라미터 튜닝

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [50]:
stopwords = pd.read_csv('./필요 데이터/NAVER_movie/stopword.txt', header = None)
stopwords = stopwords.iloc[:,0].tolist()
print(stopwords)

['않다', '되어다', '되다', '하다', '어떻다', '이렇다', '이다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다', '한다면', '등', '등등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', '얼마나', '얼마만큼', '얼마큼', '남짓', '여', '얼마간', '약간', '다소', '좀', '조금', '다수', '몇', '얼마', '지만', '하물며', '또한', '그러나', '그렇지만', '하지만', '이외에

In [54]:
pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words =  stopwords, tokenizer = tw_tokenizer)),
                     ('lr_clf', LogisticRegression(solver = 'liblinear', random_state = 0))])

In [58]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
params = {'tfidf_vect__min_df': [1, 2, 3, 4, 5],
          'tfidf_vect__max_df': [0.80, 0.85, 0.90, 0.95],
          'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
          'lr_clf__C': [1, 3.5, 4.5, 5.5, 10]}

grid_clf = GridSearchCV(pipeline, param_grid = params, 
                        cv = 3, scoring = 'accuracy', verbose = 1)

# 학습
grid_clf.fit(train_df['document'], train_df['label'])

# 최적의 하이퍼파라미터
print(grid_clf.best_params_)
print(grid_clf.best_score_)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
# 예측
pred = grid_clf.predict(test_df['document'])

# 평가
accuracy = accuracy_score(test_df['label'], pred)
print(f'정확도: {accuracy: .3f}')